#### IMPORTANT
Ensure you are utilizing 64-bit REFPROP with 64-bit python. If using the free version of REFPROP (MINI-REFPROP), please use 32-bit python and make changes to match the location where MINI-REFPROP is installed and make changes to the REFPROPFunctionLibrary function to read the REFPROP.DLL file.

Information on REFPROP and functions can be found here: https://buildmedia.readthedocs.org/media/pdf/refprop-docs/latest/refprop-docs.pdf

### IMPORT PACKAGES & FUNCTIONS

In [1]:
# Dictate the environment's loctaion of REFPROP
import os
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'

In [2]:
# Import the main class from the Python library
from ctREFPROP.ctREFPROP import REFPROPFunctionLibrary

# Imports from conda-installable packages
import pandas as pd

# Import numpy
import numpy as np

# Import matplotlib for plotting
import matplotlib.pyplot as plt

# Import Math for common values such as PI
import math


In [3]:
# Instantiate the library, and use the environment variable to explicitly state which path we want to use.
# As mentioned above, this will be changed to call the correct REFPROP functions to be used
# with MINI-REFPROP and 32-bit python.
# If using MINI-REFPROP and 32-bit python please make the following changes
# RP = REFPROPFunctionLibrary('C:/Program FIles (x86)/MINI-REFPROP\\REFPROP.DLL')
RP = REFPROPFunctionLibrary(os.environ['RPPREFIX'])

In [4]:
# This will call which root directory that will be used for the program. 
RP.SETPATHdll(os.environ['RPPREFIX'])

In [5]:
# Get the unit system we want to use (Mass base SI gives units in
# K, Pa, kg, m, N, J, W, and s)
MASS_BASE_SI = RP.GETENUMdll(0, "MASS BASE SI").iEnum

### sCO2 Loop Calculations

#### System Parameters

In [6]:
# Tube inner diameter and outer diameter
Tube_OD = 0.50 # [inch]
Tube_Thick = 0.065 # [inch]
Tube_ID = Tube_OD - 2 * Tube_Thick # inch

Tube_OD = Tube_OD * 0.0254 # [Convert inches to meters]
Tube_ID = Tube_ID * 0.0254 # [Convert inches to meters]

# Mass flow rate of sCO2
m_dot = 0.2 # [kg/s]

#### Outlet of Compressor (State  1)

In [7]:
# Temperature will be compared at end of script
T1 = 60.8 # [C]
P1 = 2900 # [psia]

T1 = T1 + 273.15 # Convert C to Kelvin
P1 = P1 * 6894.8 # convert psia to Pa

print("Pressure at Outlet of Compressor =", P1/6894.8, "psia")
print("Temperature at Outlet of Compressor =" , (T1 - 273.15) * (9/5) + 32, "F")

Pressure at Outlet of Compressor = 2900.0 psia
Temperature at Outlet of Compressor = 141.44000000000003 F


In [8]:
# Obtain fluid properties from the pressure and temperature outlined above
State_1 = RP.REFPROPdll("CO2","PT","D;V;H;S;CP/CV;W;TCX;VIS;PRANDTL", MASS_BASE_SI,0,0,P1,T1,[1.0])

# Outputs will be placed into data frame for organization
State_1 = pd.DataFrame(State_1.Output[0:9],
            index = ['Density [kg/m^3]', 'Volume [m^3/kg]', 'Enthalpy [J/kg]', 'Entropy [J/kg]',
                     'CP/CV', 'Speed of Sound', 'Thermal Cond. [W/(mK)]', 'Viscosity [Pa-s]', 'Prandtl'],
            columns = ['State 1'])

# Display the data frame
State_1

,State 1
Density [kg/m^3],718.546425
Volume [m^3/kg],0.001392
Enthalpy [J/kg],326519.585095
Entropy [J/kg],1352.393713
CP/CV,2.724135
Speed of Sound,408.530472
Thermal Cond. [W/(mK)],0.077787
Viscosity [Pa-s],0.000060
Prandtl,1.930132


#### Pressure drop towards Heat Source

In [9]:
# Find Velocity, Reynolds Number, and darcy friction factor (assuming smooth pipe)
Velocity = 4 * m_dot * State_1.loc['Volume [m^3/kg]','State 1'] / (math.pi * Tube_ID**2)
Reynolds = State_1.loc['Density [kg/m^3]','State 1'] * Velocity * Tube_ID / State_1.loc['Viscosity [Pa-s]','State 1']
Darcy_f = (0.79 * math.log(Reynolds) - 1.64)**(-2)

In [10]:
# Using Estimated Length of Tubing connecting compressor and Heat Source, 
# find the amount of pressure drop caused by fanno flow
Length = 3.71475 # [meters]

# Force acted on the wall of tube
Force = math.pi * Tube_ID * Darcy_f * State_1.loc['Density [kg/m^3]','State 1'] * (Velocity**2) * Length / 8 

# Dimensionless Friction factor
f_dim = 4 * Force / (P1 * math.pi * Tube_ID**2) 

# Inlet Mach Number of length of tubing
Mach_inlet = Velocity / State_1.loc['Speed of Sound','State 1']

# Formulation used to calculate Pressure drop (found in Adv. Fluid Mechanics Textbook)
A_eq = ((Mach_inlet**2) * (1 + ((State_1.loc['CP/CV','State 1'] - 1) / 2) * (Mach_inlet**2))) \
        / ((1 + State_1.loc['CP/CV','State 1'] * (Mach_inlet**2) - f_dim)**2)

# Either Mach numbers will be negative. Use the positive value in the following calculations
Mach_outlet_1 = math.sqrt((-1 * (1 - 2 * A_eq * State_1.loc['CP/CV','State 1'])\
                          + ((1 + 2 * A_eq * (State_1.loc['CP/CV','State 1'] + 1))**0.5))\
                         / ((State_1.loc['CP/CV','State 1'] - 1) - 2 * A_eq * State_1.loc['CP/CV','State 1']**2))
#Mach_outlet_2 = math.sqrt((-1 * (1 - 2 * A_eq * State_1.loc['CP/CV','State 1'])\
#                          - ((1 + 2 * A_eq * (State_1.loc['CP/CV','State 1'] + 1))**0.5))\
#                         / ((State_1.loc['CP/CV','State 1'] - 1) - 2 * A_eq * State_1.loc['CP/CV','State 1']**2))

# Find Outlet pressure caused by fanno flow (frictional loss)
P2 = P1 * (1 + State_1.loc['CP/CV','State 1'] * (Mach_inlet**2) - f_dim) / (1 + State_1.loc['CP/CV','State 1']\
                                                                           * (Mach_outlet_1**2))

print("Pressure at Inlet of Heat Source =", round(P2/6894.8 , 3), "psia")
print("Temperature at Inlet of Heat Source =" , (T1 - 273.15) * (9/5) + 32, "F")

Pressure at Inlet of Heat Source = 2892.272 psia
Temperature at Inlet of Heat Source = 141.44000000000003 F


#### Inlet of Heat Source (State 2a)

In [11]:
#Pressure and temperature of fluid at inlet of Heat Source
P2a = P2
T2a = T1

# Tube ID (using 1" OD Tubes)
Tube_HS_ID = .81 * 0.0254 # convert inches to meters

# Obtain fluid properties from the pressure and temperature outlined above
State_2a = RP.REFPROPdll("CO2","PT","D;V;H;S;CP/CV;W;TCX;VIS;PRANDTL", MASS_BASE_SI,0,0,P2a,T2a,[1.0])

# Outputs will be placed into data frame for organization
State_2a = pd.DataFrame(State_2a.Output[0:9],
            index = ['Density [kg/m^3]', 'Volume [m^3/kg]', 'Enthalpy [J/kg]', 'Entropy [J/kg]',
                     'CP/CV', 'Speed of Sound', 'Thermal Cond. [W/(mK)]', 'Viscosity [Pa-s]', 'Prandtl'],
            columns = ['State 2a'])

# Display the data frame
State_2a

,State 2a
Density [kg/m^3],717.673988
Volume [m^3/kg],0.001393
Enthalpy [J/kg],326664.346395
Entropy [J/kg],1353.049371
CP/CV,2.729241
Speed of Sound,407.611053
Thermal Cond. [W/(mK)],0.077682
Viscosity [Pa-s],0.000060
Prandtl,1.932877


In [12]:
# Find Velocity, Reynolds Number, and darcy friction factor (assuming smooth pipe)
Velocity = 4 * m_dot * State_2a.loc['Volume [m^3/kg]','State 2a'] / (math.pi * Tube_HS_ID**2)
Reynolds = State_2a.loc['Density [kg/m^3]','State 2a'] * Velocity * Tube_HS_ID / State_2a.loc['Viscosity [Pa-s]','State 2a']
Darcy_f = (0.79 * math.log(Reynolds) - 1.64)**(-2)

In [13]:
# Using Estimated Length of Tubing connecting compressor and Heat Source, 
# find the amount of pressure drop caused by fanno flow
Length = 19.2024 # [meters]

# Force acted on the wall of tube
Force = math.pi * Tube_HS_ID * Darcy_f * State_2a.loc['Density [kg/m^3]','State 2a'] * (Velocity**2) * Length / 8 

# Dimensionless Friction factor
f_dim = 4 * Force / (P2a * math.pi * Tube_HS_ID**2) 

# Heat Added into system (kW)
Q = 18

# Dimensionless heating factor
q_dim = Q * 1000 / (m_dot * State_2a.loc['Enthalpy [J/kg]','State 2a'])

# Inlet Mach Number of length of tubing
Mach_inlet = Velocity / State_2a.loc['Speed of Sound','State 2a']

# Formulation used to calculate Pressure drop (found in Adv. Fluid Mechanics Textbook)
A_eq = ((Mach_inlet**2) * (1 + ((State_2a.loc['CP/CV','State 2a'] - 1) / 2) * (Mach_inlet**2) + q_dim)) \
        / ((1 + State_2a.loc['CP/CV','State 2a'] * (Mach_inlet**2) - f_dim)**2)

# Either Mach numbers will be negative. Use the positive value in the following calculations
Mach_outlet_1 = math.sqrt((-1 * (1 - 2 * A_eq * State_2a.loc['CP/CV','State 2a'])\
                          + ((1 + 2 * A_eq * (State_2a.loc['CP/CV','State 2a'] + 1))**0.5))\
                         / ((State_2a.loc['CP/CV','State 2a'] - 1) - 2 * A_eq * State_2a.loc['CP/CV','State 2a']**2))
#Mach_outlet_2 = math.sqrt((-1 * (1 - 2 * A_eq * State_1.loc['CP/CV','State 1'])\
#                          - ((1 + 2 * A_eq * (State_1.loc['CP/CV','State 1'] + 1))**0.5))\
#                         / ((State_1.loc['CP/CV','State 1'] - 1) - 2 * A_eq * State_1.loc['CP/CV','State 1']**2))

# Find Outlet pressure caused by fanno flow (frictional loss)
P2b = P2a * (1 + State_2a.loc['CP/CV','State 2a'] * (Mach_inlet**2) - f_dim) / (1 + State_2a.loc['CP/CV','State 2a']\
                                                                           * (Mach_outlet_1**2))

print("Pressure at Outlet of Heat Source =", round(P2b/6894.8 , 3), "psia")

Pressure at Outlet of Heat Source = 2891.55 psia


In [14]:
# Find the enthalpy at the outlet of the Heat source
Enth_2b = (State_2a.loc['Enthalpy [J/kg]','State 2a'] * (1 + ((State_2a.loc['CP/CV','State 2a'] - 1) / 2) * Mach_inlet**2 + q_dim)) / \
            (1 + ((State_2a.loc['CP/CV','State 2a'] - 1) / 2) * (Mach_outlet_1**2))

Enth_2b # [J/kg]

416655.21169168595

#### Outlet of Heat Source (State 2b)

In [15]:
# Using the new Pressure and enthalpy find the states of the fluid at the outlet of Heat Source

State_2b = RP.REFPROPdll("CO2","PH","T;D;V;S;CP/CV;W;TCX;VIS;PRANDTL", MASS_BASE_SI,0,0,P2b,Enth_2b,[1.0])

# Outputs will be placed into data frame for organization
State_2b = pd.DataFrame(State_2b.Output[0:9],
            index = ['Temperature [K]', 'Density [kg/m^3]', 'Volume [m^3/kg]', 'Entropy [J/kg]',
                     'CP/CV', 'Speed of Sound', 'Thermal Cond. [W/(mK)]', 'Viscosity [Pa-s]', 'Prandtl'],
            columns = ['State 2b'])

# Display the data frame
State_2b

,State 2b
Temperature [K],368.988576
Density [kg/m^3],499.670747
Volume [m^3/kg],0.002001
Entropy [J/kg],1609.409486
CP/CV,2.656505
Speed of Sound,317.117582
Thermal Cond. [W/(mK)],0.056640
Viscosity [Pa-s],0.000038
Prandtl,1.621379


In [16]:
# Add enthalpy to the data frame
State_2b.loc['Enthalpy [J/kg]', 'State 2b'] = Enth_2b

In [17]:
# Find Velocity, Reynolds Number, and darcy friction factor (assuming smooth pipe)
Velocity = 4 * m_dot * State_2b.loc['Volume [m^3/kg]','State 2b'] / (math.pi * Tube_ID**2)
Reynolds = State_2b.loc['Density [kg/m^3]','State 2b'] * Velocity * Tube_ID / State_2b.loc['Viscosity [Pa-s]','State 2b']
Darcy_f = (0.79 * math.log(Reynolds) - 1.64)**(-2)

In [18]:
# Using Estimated Length of Tubing connecting Heat Source and Engine 
# find the amount of pressure drop caused by fanno flow
Length = 2.286 # [meters]

# Force acted on the wall of tube
Force = math.pi * Tube_ID * Darcy_f * State_2b.loc['Density [kg/m^3]','State 2b'] * (Velocity**2) * Length / 8 

# Dimensionless Friction factor
f_dim = 4 * Force / (P2b * math.pi * Tube_ID**2) 

# Heat Added into system (kW)
Q = 0

# Dimensionless heating factor
q_dim = Q * 1000 / (m_dot * State_2b.loc['Enthalpy [J/kg]','State 2b'])

# Inlet Mach Number of length of tubing
Mach_inlet = Velocity / State_2b.loc['Speed of Sound','State 2b']

# Formulation used to calculate Pressure drop (found in Adv. Fluid Mechanics Textbook)
A_eq = ((Mach_inlet**2) * (1 + ((State_2b.loc['CP/CV','State 2b'] - 1) / 2) * (Mach_inlet**2) + q_dim)) \
        / ((1 + State_2b.loc['CP/CV','State 2b'] * (Mach_inlet**2) - f_dim)**2)

# Either Mach numbers will be negative. Use the positive value in the following calculations
Mach_outlet_1 = math.sqrt((-1 * (1 - 2 * A_eq * State_2b.loc['CP/CV','State 2b'])\
                          + ((1 + 2 * A_eq * (State_2b.loc['CP/CV','State 2b'] + 1))**0.5))\
                         / ((State_2b.loc['CP/CV','State 2b'] - 1) - 2 * A_eq * State_2b.loc['CP/CV','State 2b']**2))
#Mach_outlet_2 = math.sqrt((-1 * (1 - 2 * A_eq * State_1.loc['CP/CV','State 1'])\
#                          - ((1 + 2 * A_eq * (State_1.loc['CP/CV','State 1'] + 1))**0.5))\
#                         / ((State_1.loc['CP/CV','State 1'] - 1) - 2 * A_eq * State_1.loc['CP/CV','State 1']**2))

# Find Outlet pressure caused by fanno flow (frictional loss)
P3 = P2b * (1 + State_2b.loc['CP/CV','State 2b'] * (Mach_inlet**2) - f_dim) / (1 + State_2b.loc['CP/CV','State 2b']\
                                                                           * (Mach_outlet_1**2))

print("Pressure at Inlet of Engine =", round(P3/6894.8 , 3), "psia")

Pressure at Inlet of Engine = 2876.727 psia


In [ ]:
# Find the enthalpy at the inlet of the Engine
Enth_2b = (State_2a.loc['Enthalpy [J/kg]','State 2a'] * (1 + ((State_2a.loc['CP/CV','State 2a'] - 1) / 2) * Mach_inlet**2 + q_dim)) / \
            (1 + ((State_2a.loc['CP/CV','State 2a'] - 1) / 2) * (Mach_outlet_1**2))

Enth_2b # [J/kg]

#### Inlet of Engine

In [19]:
#Pressure and temperature of fluid at inlet of Engine
# P3 is found above
T3 = State_2b.loc['Temperature [K]', 'State 2b']  # Assume temperature difference from outlet of Heat source to inlet of engine is negligible

# Obtain fluid properties from the pressure and temperature outlined above
State_3 = RP.REFPROPdll("CO2","PT","D;V;H;S;CP/CV;W;TCX;VIS;PRANDTL", MASS_BASE_SI,0,0,P3,T3,[1.0])

# Outputs will be placed into data frame for organization
State_3 = pd.DataFrame(State_3.Output[0:9],
            index = ['Density [kg/m^3]', 'Volume [m^3/kg]', 'Enthalpy [J/kg]', 'Entropy [J/kg]',
                     'CP/CV', 'Speed of Sound', 'Thermal Cond. [W/(mK)]', 'Viscosity [Pa-s]', 'Prandtl'],
            columns = ['State 3'])

# Display the data frame
State_3

,State 3
Density [kg/m^3],496.959469
Volume [m^3/kg],0.002012
Enthalpy [J/kg],417238.166450
Entropy [J/kg],1611.545196
CP/CV,2.659031
Speed of Sound,315.930921
Thermal Cond. [W/(mK)],0.056409
Viscosity [Pa-s],0.000038
Prandtl,1.620345
